In [5]:
import pandas as pd
import numpy as np
from pathlib import Path
import csv
import os
from googletrans import Translator
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
import plotly.graph_objects as go

def Delimitador (path):
    data = open(path, "r", encoding="latin_1").read()
    delimiter = csv.Sniffer().sniff(data).delimiter
    return delimiter  

def archivos (path):
    archs = os.walk(path)
    for i,e,archivo in archs:
        return archivo

def DataFrame (path, nombre):
    datas = archivos(path)
    for i in datas:
        direc = path + "/" + i
        deli = Delimitador(direc)

        if i == "energyco2.csv" and nombre == "energy":
            df = pd.read_csv(direc,delimiter=deli,encoding="latin_1", index_col=False)
            return df

        if i == "global_power_plant_database.csv" and nombre == "global":
            df_global = pd.read_csv(direc,delimiter=",", encoding="latin1")
            return df_global

        if i == "owid-energy-consumption-source.csv" and nombre == "owid1":
            df_owid1 = pd.read_csv(direc,delimiter=deli,encoding="latin_1")
            return df_owid1

        if i == "owid-energy-datadiccionary.csv" and nombre == "owid2":
            df_owi2 = pd.read_csv(direc,delimiter=deli,encoding="latin_1")
            return df_owi2

def limpieza (df):
    translator = Translator()
    for i in df.columns:
        df.rename(columns={i:translator.translate(i, dest="es").text.title()}, inplace=True)
    df.drop(columns="Sin Nombre: 0", inplace=True)

    otro = pd.read_csv("./owid-co2-data.csv", delimiter=",", encoding="utf-8")
    otro = otro[["year","country","population","gdp"]]
    df = pd.merge(df, otro, left_on=["Año","País"], right_on=["year","country"], how="left")
    df.loc[(df["Población"].isna() == True) & (df["population"].isna() == False), "Población"] = df.loc[(df["Población"].isna() == True) & (df["population"].isna() == False)]["population"]
    df.loc[(df["Pib"].isna() == True) & (df["gdp"].isna() == False), "Pib"] = df.loc[(df["Pib"].isna() == True) & (df["gdp"].isna() == False)]["gdp"]
    df.drop(columns=["country", "year", "population","gdp"], inplace=True)

    df.dropna(subset=["Emisión De Co2"], inplace=True)

    for i in df.columns:
        if i == "Tipo_Energia":
            continue
        elif "Energ" in i:
            nom = i + " " + "TWh"
            df[nom] = df[i] / 3.412e+12

    return df



if __name__ == "__main__":
    directorio = os.getcwd()
    df_energy = DataFrame(directorio, "energy")
    df_energy = limpieza(df_energy)

In [6]:
df_energy

,País,Tipo_Energia,Año,Consumo De Energía,Producción De Energía,Pib,Población,Intensidad_Energética_Per_Capita,Intensidad_Energética_Por_Pib,Emisión De Co2,Consumo De Energía TWh,Producción De Energía TWh,Intensidad_Energética_Per_Capita TWh,Intensidad_Energética_Por_Pib TWh
0,World,all_energy_types,1980,292.899790,296.337228,27770.910281,4.298127e+06,68.145921,10.547000,4946.627130,8.584402e-11,8.685147e-11,1.997243e-11,3.091149e-12
1,World,coal,1980,78.656134,80.114194,27770.910281,4.298127e+06,68.145921,10.547000,1409.790188,2.305279e-11,2.348013e-11,1.997243e-11,3.091149e-12
2,World,natural_gas,1980,53.865223,54.761046,27770.910281,4.298127e+06,68.145921,10.547000,1081.593377,1.578699e-11,1.604954e-11,1.997243e-11,3.091149e-12
3,World,petroleum_n_other_liquids,1980,132.064019,133.111109,27770.910281,4.298127e+06,68.145921,10.547000,2455.243565,3.870575e-11,3.901263e-11,1.997243e-11,3.091149e-12
4,World,nuclear,1980,7.575700,7.575700,27770.910281,4.298127e+06,68.145921,10.547000,0.000000,2.220311e-12,2.220311e-12,1.997243e-11,3.091149e-12
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
55435,Zimbabwe,coal,2019,0.045064,0.075963,37.620400,1.465420e+04,11.508701,4.482962,4.586869,1.320741e-14,2.226347e-14,3.373007e-12,1.313881e-12
55436,Zimbabwe,natural_gas,2019,0.000000,0.000000,37.620400,1.465420e+04,11.508701,4.482962,0.000000,0.000000e+00,0.000000e+00,3.373007e-12,1.313881e-12
55437,Zimbabwe,petroleum_n_other_liquids,2019,0.055498,0.000000,37.620400,1.465420e+04,11.508701,4.482962,4.377890,1.626551e-14,0.000000e+00,3.373007e-12,1.313881e-12
55438,Zimbabwe,nuclear,2019,NaN,NaN,37.620400,1.465420e+04,11.508701,4.482962,0.000000,NaN,NaN,3.373007e-12,1.313881e-12


In [7]:
def mapPlot(df_newmap):
    df_newmap = df_newmap[["País", "Año", "Tipo_Energia", "Emisión De Co2"]]
    df_newmap = df_newmap[(df_newmap["País"] != "World") & (df_newmap["Tipo_Energia"] == "all_energy_types") & (df_newmap["Año"] == 2019)]

    for i in df_newmap["País"].values:
        if "U.S." in i:
            aux = df_newmap[df_newmap["País"] == i].index
            df_newmap = df_newmap.drop(aux)

    data = dict(type = 'choropleth', 
                locations = df_newmap['País'], 
                locationmode = 'country names', 
                z = list(df_newmap['Emisión De Co2']), 
                colorscale=[
                    [0, "rgb(243, 255, 250)"],
                    [0.10, "rgb(166, 255, 249)"],
                    [0.25, "rgb(132, 255, 233)"],
                    [0.50, "rgb(100, 255, 233)"],
                    [0.60, "rgb(0, 248, 248)"],
                    [0.75, "rgb(0, 186, 186)"],
                    [1, "rgb(0, 124, 124)"]],
                text = df_newmap['País'])
    layout = dict(geo = dict(scope='world'))

    fig = go.Figure(data=[data], layout=layout)
    fig.update_layout(
        # title='Mapa de fallecidos por covid',
        # width=800,
        margin=dict(l=5,r=5,b=5,t=5),
        paper_bgcolor= 'black',
        font_color='#cee3e1',
    )              
    return fig

In [8]:
mapPlot(df_energy)

In [9]:
def linealPlot(df_new):
    df_new = df_new[(df_new["País"] == "World") & (df_new["Tipo_Energia"] == "all_energy_types")]
    df_new = df_new[["País", "Año", "Emisión De Co2"]]
    fig= go.Figure(data=[
        go.Scatter(
        x=df_new['Año'],
        y=df_new['Emisión De Co2'],
        mode='lines', 
        name='Emisiones Co2',
        line=dict(color='cyan')
        )
    ])
    fig.update_layout(
        height=300, width=1000,
        plot_bgcolor='black',
        paper_bgcolor= 'black',
        font_color='#cee3e1',
        legend=dict(
            x=0.05,
            y=1,
            title_font_family="Times New Roman",
            font=dict(
                family="Courier",
                size=12,
                color="LightSteelBlue"
            ),
            bgcolor="Black",
            bordercolor="LightSteelBlue",
            borderwidth=1
        ),
        xaxis=dict(showgrid=False,showline=True,linecolor='rgb(255,255,255)'),
        yaxis=dict(showgrid=False),
        margin=dict(l=10,r=10,b=10,t=10)
    )
    return fig


In [10]:
linealPlot(df_energy)